In [ ]:
import pandas as pd
import numpy as np
from pyproj import Transformer
from scipy.spatial import KDTree
import plotly.graph_objs as go

# Step 1: Load the satellite data from the .gmd file
file_path = "path_to_your_file.gmd"  # Replace with your actual file path
columns = ['Timestamp', 'MeasurementType', 'SatelliteID', 'AdditionalID', 'X', 'Y', 'Z']
df = pd.read_csv(file_path, sep='\s+', names=columns)

# Step 2: Convert X, Y, Z from kilometers to meters
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * 1000  # Ensure correct scaling to meters

# Step 3: Convert GPS timestamp to seconds (assume fractional days)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) * 86400  # Days to seconds

# Step 4: Group points by space-time proximity
spatial_threshold = 10000  # 10 km in meters
time_threshold = 30  # 30 seconds

df = df.sort_values('Timestamp').reset_index(drop=True)
points = df[['X', 'Y', 'Z']].values
tree = KDTree(points)

groups = []
visited = set()

for i in range(len(df)):
    if i not in visited:
        group = [i]
        visited.add(i)

        for j in range(i + 1, len(df)):
            if j not in visited:
                if np.linalg.norm(points[i] - points[j]) <= spatial_threshold:
                    if abs(df.iloc[j]['Timestamp'] - df.iloc[group[-1]]['Timestamp']) <= time_threshold:
                        group.append(j)
                        visited.add(j)

        groups.append(group)

# Step 5: Create Earth sphere
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

earth_radius = 6371e3  # Earth's mean radius in meters
x_sphere = earth_radius * np.sin(theta) * np.cos(phi)
y_sphere = earth_radius * np.sin(theta) * np.sin(phi)
z_sphere = earth_radius * np.cos(theta)

earth = go.Surface(
    x=x_sphere, y=y_sphere, z=z_sphere,
    opacity=0.5, colorscale='Blues', showscale=False, name='Earth'
)

# Step 6: Create frames for animation
frames = [
    go.Frame(
        data=[go.Scatter3d(
            x=df.iloc[group]['X'], y=df.iloc[group]['Y'], z=df.iloc[group]['Z'],
            mode='markers',  # Only markers, no lines
            marker=dict(size=5, color='red', opacity=0.8),
            name=f"Arc {i}"
        )],
        name=f"Arc {i}"
    )
    for i, group in enumerate(groups)
]

# Step 7: Layout configuration with the Earth sphere visible at all times
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-7e6, 7e6]),
        yaxis=dict(title='Y (meters)', range=[-7e6, 7e6]),
        zaxis=dict(title='Z (meters)', range=[-7e6, 7e6]),
        aspectmode='data'
    ),
    title='Satellite Arcs with Disjoint Groups on Earth Sphere',
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(
            label="Play", method="animate",
            args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True, mode='immediate')]
        )]
    )],
    sliders=[dict(
        steps=[dict(method='animate', args=[[f"Arc {i}"], dict(mode='immediate')], label=f"Arc {i}")
               for i in range(len(groups))],
        active=0,
        x=0.1, y=0, len=0.8
    )]
)

# Step 8: Create the figure with the Earth sphere and frames
fig = go.Figure(data=[earth], layout=layout, frames=frames)

# Step 9: Save and open the plot
fig.write_html("satellite_arcs_no_linkage_corrected.html", auto_open=True)

In [2]:
import pandas as pd
import numpy as np
from pyproj import Transformer
from scipy.spatial import KDTree
import plotly.graph_objs as go

# Step 1: Load the satellite data from the .gmd file
file_path = "gmat_gps.gmd"  # Replace with your actual file path
columns = ['Timestamp', 'MeasurementType', 'SatelliteID', 'AdditionalID', 'X', 'Y', 'Z']
df = pd.read_csv(file_path, sep='\s+', names=columns)

# Step 2: Convert X, Y, Z from kilometers to meters
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * 1000  # Ensure correct scaling to meters

# Step 3: Convert GPS timestamp to seconds (assume fractional days)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) * 86400  # Days to seconds

# Step 4: Group points by space-time proximity
spatial_threshold = 10000*300  # 10 km in meters
time_threshold = 3600*8  # 30 seconds

df = df.sort_values('Timestamp').reset_index(drop=True)
points = df[['X', 'Y', 'Z']].values
tree = KDTree(points)

groups = []
visited = set()

for i in range(len(df)):
    if i not in visited:
        group = [i]
        visited.add(i)

        for j in range(i + 1, len(df)):
            if j not in visited:
                if np.linalg.norm(points[i] - points[j]) <= spatial_threshold:
                    if abs(df.iloc[j]['Timestamp'] - df.iloc[group[-1]]['Timestamp']) <= time_threshold:
                        group.append(j)
                        visited.add(j)

        groups.append(group)

# Step 5: Create Earth sphere
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

earth_radius = 6371e3  # Earth's mean radius in meters
x_sphere = earth_radius * np.sin(theta) * np.cos(phi)
y_sphere = earth_radius * np.sin(theta) * np.sin(phi)
z_sphere = earth_radius * np.cos(theta)

earth = go.Surface(
    x=x_sphere, y=y_sphere, z=z_sphere,
    opacity=0.5, colorscale='Blues', showscale=False, name='Earth'
)

# Step 6: Create frames for animation
frames = [
    go.Frame(
        data=[go.Scatter3d(
            x=df.iloc[group]['X'], y=df.iloc[group]['Y'], z=df.iloc[group]['Z'],
            mode='markers',  # Only markers, no lines
            marker=dict(size=5, color='red', opacity=0.8),
            name=f"Arc {i}"
        )],
        name=f"Arc {i}"
    )
    for i, group in enumerate(groups)
]

# Step 7: Layout configuration with the Earth sphere visible at all times
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-7e6, 7e6]),
        yaxis=dict(title='Y (meters)', range=[-7e6, 7e6]),
        zaxis=dict(title='Z (meters)', range=[-7e6, 7e6]),
        aspectmode='data'
    ),
    title='Satellite Arcs with Disjoint Groups on Earth Sphere',
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(
            label="Play", method="animate",
            args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True, mode='immediate')]
        )]
    )],
    sliders=[dict(
        steps=[dict(method='animate', args=[[f"Arc {i}"], dict(mode='immediate')], label=f"Arc {i}")
               for i in range(len(groups))],
        active=0,
        x=0.1, y=0, len=0.8
    )]
)

# Step 8: Create the figure with the Earth sphere and frames
fig = go.Figure(data=[earth], layout=layout, frames=frames)

# Step 9: Save and open the plot
fig.write_html("satellite_arcs_no_linkage_corrected.html", auto_open=True)

In [7]:
import pandas as pd
import numpy as np
from pyproj import Transformer
from scipy.spatial import KDTree
import plotly.graph_objs as go

# Step 1: Load the satellite data from the .gmd file
file_path = "gmat_gps.gmd"  # Replace with your actual file path
columns = ['Timestamp', 'MeasurementType', 'SatelliteID', 'AdditionalID', 'X', 'Y', 'Z']
df = pd.read_csv(file_path, sep='\s+', names=columns)

# Step 2: Convert X, Y, Z from kilometers to meters
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * 1000  # Ensure correct scaling to meters

# Step 3: Convert GPS timestamp to seconds (assume fractional days)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) * 86400  # Convert days to seconds

# Step 4: Time-first grouping: Group points by time windows first, then by spatial proximity
time_threshold = 3600*9  # 1 hour threshold in seconds
spatial_threshold = 1000*5000  # 10 km threshold in meters

# Step 4.1: Group points by time (first priority)
time_groups = []
visited_time = set()

for i in range(len(df)):
    if i not in visited_time:
        # Create a new time-based group starting with point 'i'
        current_group = [i]
        visited_time.add(i)

        # Add all points within the time threshold
        for j in range(i + 1, len(df)):
            if j not in visited_time and abs(df.iloc[j]['Timestamp'] - df.iloc[i]['Timestamp']) <= time_threshold:
                current_group.append(j)
                visited_time.add(j)

        time_groups.append(current_group)

# Step 4.2: Within each time group, further group by spatial proximity
final_groups = []
for time_group in time_groups:
    group_points = df.iloc[time_group][['X', 'Y', 'Z']].values
    tree = KDTree(group_points)

    visited_space = set()
    for i, point in enumerate(group_points):
        if i not in visited_space:
            # Create a spatial group starting with the current point
            spatial_group = [time_group[i]]
            visited_space.add(i)

            # Find all nearby points within the spatial threshold
            neighbors = tree.query_ball_point(point, r=spatial_threshold)
            for neighbor in neighbors:
                if neighbor not in visited_space:
                    spatial_group.append(time_group[neighbor])
                    visited_space.add(neighbor)

            final_groups.append(spatial_group)


final_groups = time_group

# Step 5: Create Earth sphere
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

earth_radius = 6371e3  # Earth's mean radius in meters
x_sphere = earth_radius * np.sin(theta) * np.cos(phi)
y_sphere = earth_radius * np.sin(theta) * np.sin(phi)
z_sphere = earth_radius * np.cos(theta)

earth = go.Surface(
    x=x_sphere, y=y_sphere, z=z_sphere,
    opacity=0.5, colorscale='Blues', showscale=False, name='Earth'
)

# Step 6: Create frames for animation (no linkage between arcs)
frames = [
    go.Frame(
        data=[go.Scatter3d(
            x=df.iloc[group]['X'], y=df.iloc[group]['Y'], z=df.iloc[group]['Z'],
            mode='markers',  # Only markers, no lines
            marker=dict(size=5, color='red', opacity=0.8),
            name=f"Arc {i}"
        )],
        name=f"Arc {i}"
    )
    for i, group in enumerate(final_groups)
]

# Step 7: Layout configuration with the Earth sphere visible at all times
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-7e6, 7e6]),
        yaxis=dict(title='Y (meters)', range=[-7e6, 7e6]),
        zaxis=dict(title='Z (meters)', range=[-7e6, 7e6]),
        aspectmode='data'
    ),
    title='Satellite Arcs Grouped by Time and Space on Earth Sphere',
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(
            label="Play", method="animate",
            args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True, mode='immediate')]
        )]
    )],
    sliders=[dict(
        steps=[dict(method='animate', args=[[f"Arc {i}"], dict(mode='immediate')], label=f"Arc {i}")
               for i in range(len(final_groups))],
        active=0,
        x=0.1, y=0, len=0.8
    )]
)

# Step 8: Create the figure with the Earth sphere and frames
fig = go.Figure(data=[earth], layout=layout, frames=frames)

# Step 9: Save and open the plot
fig.write_html("satellite_arcs_time_first.html", auto_open=True)


ValueError: 
    Invalid value of type 'numpy.float64' received for the 'x' property of scatter3d
        Received value: -3458168.1925995667

    The 'x' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series

In [13]:
import pandas as pd
import numpy as np
from pyproj import Transformer
import plotly.graph_objs as go

# Step 1: Load the satellite data from the .gmd file
file_path = "gmat_gps.gmd"  # Replace with your actual file path
columns = ['Timestamp', 'MeasurementType', 'SatelliteID', 'AdditionalID', 'X', 'Y', 'Z']
df = pd.read_csv(file_path, sep='\s+', names=columns)

# Step 2: Convert X, Y, Z from kilometers to meters
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * 1000  # Scale to meters

# Step 3: Convert GPS timestamp to seconds (assume fractional days)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) * 86400  # Convert days to seconds

# Step 4: Group points by time windows only
time_threshold = 3600*2  # 1 hour in seconds

# Initialize groups
groups = []
visited = set()

# Group by time
for i in range(len(df)):
    if i not in visited:
        # Create a new time-based group
        group = [i]
        visited.add(i)

        # Add all points within the time threshold
        for j in range(i + 1, len(df)):
            if j not in visited and abs(df.iloc[j]['Timestamp'] - df.iloc[i]['Timestamp']) <= time_threshold:
                group.append(j)
                visited.add(j)

        groups.append(group)

# Step 5: Create Earth sphere
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

earth_radius = 6371e3  # Earth's mean radius in meters
x_sphere = earth_radius * np.sin(theta) * np.cos(phi)
y_sphere = earth_radius * np.sin(theta) * np.sin(phi)
z_sphere = earth_radius * np.cos(theta)

earth = go.Surface(
    x=x_sphere, y=y_sphere, z=z_sphere,
    opacity=0.5, colorscale='Blues', showscale=False, name='Earth'
)

# Step 6: Create frames for animation (no linkage between time groups)
frames = [
    go.Frame(
        data=[go.Scatter3d(
            x=df.iloc[group]['X'], y=df.iloc[group]['Y'], z=df.iloc[group]['Z'],
            mode='markers',  # Markers only, no lines
            marker=dict(size=5, color='red', opacity=0.8),
            name=f"Time Group {i}"
        )],
        name=f"Time Group {i}"
    )
    for i, group in enumerate(groups)
]

# Step 7: Layout configuration with Earth sphere visible at all times
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-7e6, 7e6]),
        yaxis=dict(title='Y (meters)', range=[-7e6, 7e6]),
        zaxis=dict(title='Z (meters)', range=[-7e6, 7e6]),
        aspectmode='data'
    ),
    title='Satellite Points Grouped by Time on Earth Sphere',
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(
            label="Play", method="animate",
            args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True, mode='immediate')]
        )]
    )],
    sliders=[dict(
        steps=[dict(method='animate', args=[[f"Time Group {i}"], dict(mode='immediate')], label=f"Group {i}")
               for i in range(len(groups))],
        active=0,
        x=0.1, y=0, len=0.8
    )]
)

# Step 8: Create the figure with the Earth sphere and frames
fig = go.Figure(data=[earth], layout=layout, frames=frames)

# Step 9: Save and open the plot
fig.write_html("satellite_time_grouped.html", auto_open=True)

In [14]:
import pandas as pd
import numpy as np
#from pyproj import Transformer
import plotly.graph_objs as go

# Step 1: Load the satellite data from the .gmd file
file_path = "gmat_gps.gmd"  # Replace with your actual file path
columns = ['Timestamp', 'MeasurementType', 'SatelliteID', 'AdditionalID', 'X', 'Y', 'Z']
df = pd.read_csv(file_path, sep='\s+', names=columns)

# Step 2: Convert X, Y, Z from kilometers to meters
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * 1000  # Scale to meters

# Step 3: Convert GPS timestamp to seconds (assume fractional days)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) * 86400  # Convert days to seconds

# Step 4: Group points by time window (1-hour threshold)
time_threshold = 3600  # 1 hour in seconds

groups = []
visited = set()

for i in range(len(df)):
    if i not in visited:
        group = [i]
        visited.add(i)

        for j in range(i + 1, len(df)):
            if j not in visited and abs(df.iloc[j]['Timestamp'] - df.iloc[i]['Timestamp']) <= time_threshold:
                group.append(j)
                visited.add(j)

        groups.append(group)

# Step 5: Define a function to project points onto the Earth sphere surface
def project_to_ecef(x, y, z, radius=6371e3):
    norm = np.sqrt(x**2 + y**2 + z**2)
    return (x / norm) * radius, (y / norm) * radius, (z / norm) * radius

# Step 6: Create the Earth sphere for reference
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

earth_radius = 6371e3  # Earth's mean radius in meters
x_sphere = earth_radius * np.sin(theta) * np.cos(phi)
y_sphere = earth_radius * np.sin(theta) * np.sin(phi)
z_sphere = earth_radius * np.cos(theta)

earth = go.Surface(
    x=x_sphere, y=y_sphere, z=z_sphere,
    opacity=0.3, colorscale='Blues', showscale=False, name='Earth'
)

# Step 7: Create frames for animation with projections on the sphere
frames = []

for i, group in enumerate(groups):
    # Original points
    group_data = df.iloc[group]
    x, y, z = group_data['X'], group_data['Y'], group_data['Z']

    # Projected points on the ECEF sphere
    proj_x, proj_y, proj_z = project_to_ecef(x, y, z)

    # Create a frame with both original and projected points
    frame_data = [
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=5, color='red', opacity=0.8),
            name=f"Original Group {i}"
        ),
        go.Scatter3d(
            x=proj_x, y=proj_y, z=proj_z,
            mode='markers',
            marker=dict(size=4, color='green', opacity=0.6),
            name=f"Projected Group {i}"
        )
    ]

    frames.append(go.Frame(data=frame_data, name=f"Group {i}"))

# Step 8: Layout configuration with Earth sphere visible at all times
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-7e6, 7e6]),
        yaxis=dict(title='Y (meters)', range=[-7e6, 7e6]),
        zaxis=dict(title='Z (meters)', range=[-7e6, 7e6]),
        aspectmode='data'
    ),
    title='Satellite Points and ECEF Projections on Earth Sphere',
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(
            label="Play", method="animate",
            args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True, mode='immediate')]
        )]
    )],
    sliders=[dict(
        steps=[dict(method='animate', args=[[f"Group {i}"], dict(mode='immediate')], label=f"Group {i}")
               for i in range(len(groups))],
        active=0,
        x=0.1, y=0, len=0.8
    )]
)

# Step 9: Create the figure with the Earth sphere and frames
fig = go.Figure(data=[earth], layout=layout, frames=frames)

# Step 10: Save and open the plot
fig.write_html("satellite_ecef_projection.html", auto_open=True)

In [4]:
import pandas as pd
import numpy as np
#from pyproj import Transformer
import plotly.graph_objs as go

# Step 1: Load the satellite data from the .gmd file
file_path = "gmat_gps.gmd"  # Replace with your actual file path
columns = ['Timestamp', 'MeasurementType', 'SatelliteID', 'AdditionalID', 'X', 'Y', 'Z']
df = pd.read_csv(file_path, sep='\s+', names=columns)

# Step 2: Convert X, Y, Z from kilometers to meters
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * 1000  # Scale to meters

# Step 3: Convert GPS timestamp to seconds (assuming fractional days)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) * 86400  # Convert days to seconds

# Step 4: Group points by time window (1-hour threshold)
time_threshold = 3600  # 1 hour in seconds

groups = []
visited = set()

for i in range(len(df)):
    if i not in visited:
        group = [i]
        visited.add(i)

        for j in range(i + 1, len(df)):
            if j not in visited and abs(df.iloc[j]['Timestamp'] - df.iloc[i]['Timestamp']) <= time_threshold:
                group.append(j)
                visited.add(j)

        groups.append(group)

# Step 5: Define a function to project points onto the Earth sphere surface
def project_to_ecef(x, y, z, radius=6371e3):
    norm = np.sqrt(x**2 + y**2 + z**2)
    return (x / norm) * radius, (y / norm) * radius, (z / norm) * radius

# Step 6: Create the Earth sphere for reference
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

earth_radius = 6371e3  # Earth's mean radius in meters
x_sphere = earth_radius * np.sin(theta) * np.cos(phi)
y_sphere = earth_radius * np.sin(theta) * np.sin(phi)
z_sphere = earth_radius * np.cos(theta)

# Function to generate the Earth sphere for each frame
def get_earth_surface():
    return go.Surface(
        x=x_sphere, y=y_sphere, z=z_sphere,
        opacity=0.3, colorscale='Blues', showscale=False, name='Earth'
    )

# Step 7: Create frames for animation with projections on the sphere
frames = []

for i, group in enumerate(groups):
    # Original points
    group_data = df.iloc[group]
    x, y, z = group_data['X'], group_data['Y'], group_data['Z']

    # Projected points on the ECEF sphere
    proj_x, proj_y, proj_z = project_to_ecef(x, y, z)

    # Create a frame with both original and projected points plus the Earth sphere
    frame_data = [
        get_earth_surface(),  # Add the Earth sphere to each frame
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=5, color='red', opacity=0.8),
            name=f"Original Group {i}"
        ),
        go.Scatter3d(
            x=proj_x, y=proj_y, z=proj_z,
            mode='markers',
            marker=dict(size=4, color='green', opacity=0.6),
            name=f"Projected Group {i}"
        )
    ]

    frames.append(go.Frame(data=frame_data, name=f"Group {i}"))

# Step 8: Layout configuration with Earth sphere visible at all times
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-7e6, 7e6]),
        yaxis=dict(title='Y (meters)', range=[-7e6, 7e6]),
        zaxis=dict(title='Z (meters)', range=[-7e6, 7e6]),
        aspectmode='data'
    ),
    title='Satellite Points and ECEF Projections on Earth Sphere',
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(
            label="Play", method="animate",
            args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True, mode='immediate')]
        )]
    )],
    sliders=[dict(
        steps=[dict(method='animate', args=[[f"Group {i}"], dict(mode='immediate')], label=f"Group {i}")
               for i in range(len(groups))],
        active=0,
        x=0.1, y=0, len=0.8
    )]
)

# Step 9: Create the figure with the Earth sphere and frames
fig = go.Figure(data=[get_earth_surface()], layout=layout, frames=frames)

# Step 10: Save and open the plot
fig.write_html("satellite_ecef_projection_with_sphere.html", auto_open=True)

In [2]:
import pandas as pd
import numpy as np
#from pyproj import Transformer
import plotly.graph_objs as go

# Step 1: Load the satellite data from the .gmd file
file_path = "gmat_gps.gmd"  # Replace with your actual file path
columns = ['Timestamp', 'MeasurementType', 'SatelliteID', 'AdditionalID', 'X', 'Y', 'Z']
df = pd.read_csv(file_path, sep='\s+', names=columns)

# Step 2: Convert X, Y, Z from kilometers to meters
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] * 1000  # Scale to meters

# Step 3: Convert GPS timestamp to seconds (assuming fractional days)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) * 86400  # Convert days to seconds

# Step 4: Group points by time window (1-hour threshold)
time_threshold = 3600  # 1 hour in seconds

groups = []
visited = set()

for i in range(len(df)):
    if i not in visited:
        group = [i]
        visited.add(i)

        for j in range(i + 1, len(df)):
            if j not in visited and abs(df.iloc[j]['Timestamp'] - df.iloc[i]['Timestamp']) <= time_threshold:
                group.append(j)
                visited.add(j)

        groups.append(group)

# Step 5: Define a function to project points onto the Earth sphere surface
def project_to_ecef(x, y, z, radius=6371e3):
    norm = np.sqrt(x**2 + y**2 + z**2)
    return (x / norm) * radius, (y / norm) * radius, (z / norm) * radius

# Step 6: Create the Earth sphere for reference
theta = np.linspace(0, np.pi, 50)
phi = np.linspace(0, 2 * np.pi, 50)
theta, phi = np.meshgrid(theta, phi)

earth_radius = 6371e3  # Earth's mean radius in meters
x_sphere = earth_radius * np.sin(theta) * np.cos(phi)
y_sphere = earth_radius * np.sin(theta) * np.sin(phi)
z_sphere = earth_radius * np.cos(theta)

# Function to generate the Earth sphere for each frame
def get_earth_surface():
    return go.Surface(
        x=x_sphere, y=y_sphere, z=z_sphere,
        opacity=0.3, colorscale='Blues', showscale=False, name='Earth'
    )

# Step 7: Create frames for animation with projections on the sphere
frames = []

for i, group in enumerate(groups):
    # Extract group data
    group_data = df.iloc[group]
    x, y, z = group_data['X'], group_data['Y'], group_data['Z']

    # Projected points on the ECEF sphere
    proj_x, proj_y, proj_z = project_to_ecef(x, y, z)

    # Debugging: Print coordinates to ensure points are loaded correctly
    print(f"Group {i}: Original Points -> {len(x)} points")
    print(f"Group {i}: Projected Points -> {len(proj_x)} points")

    # Create a frame with both original and projected points plus the Earth sphere
    frame_data = [
        get_earth_surface(),  # Add the Earth sphere to each frame
        go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(size=5, color='red', opacity=0.8),
            name=f"Original Group {i}"
        ),
        go.Scatter3d(
            x=proj_x, y=proj_y, z=proj_z,
            mode='markers',
            marker=dict(size=4, color='green', opacity=0.6),
            name=f"Projected Group {i}"
        )
    ]

    frames.append(go.Frame(data=frame_data, name=f"Group {i}"))

# Step 8: Layout configuration with Earth sphere and correct axis ranges
layout = go.Layout(
    scene=dict(
        xaxis=dict(title='X (meters)', range=[-7e6, 7e6]),
        yaxis=dict(title='Y (meters)', range=[-7e6, 7e6]),
        zaxis=dict(title='Z (meters)', range=[-7e6, 7e6]),
        aspectmode='data'
    ),
    title='Satellite Points and ECEF Projections on Earth Sphere',
    updatemenus=[dict(
        type="buttons",
        showactive=False,
        buttons=[dict(
            label="Play", method="animate",
            args=[None, dict(frame=dict(duration=1000, redraw=True), fromcurrent=True, mode='immediate')]
        )]
    )],
    sliders=[dict(
        steps=[dict(method='animate', args=[[f"Group {i}"], dict(mode='immediate')], label=f"Group {i}")
               for i in range(len(groups))],
        active=0,
        x=0.1, y=0, len=0.8
    )]
)

# Step 9: Create the figure with the Earth sphere and frames
fig = go.Figure(data=[get_earth_surface()], layout=layout, frames=frames)

# Step 10: Save and open the plot
fig.write_html("satellite_ecef_projection_debugged.html", auto_open=True)

Group 0: Original Points -> 1 points
Group 0: Projected Points -> 1 points
Group 1: Original Points -> 1 points
Group 1: Projected Points -> 1 points
Group 2: Original Points -> 1 points
Group 2: Projected Points -> 1 points
Group 3: Original Points -> 4 points
Group 3: Projected Points -> 4 points
Group 4: Original Points -> 5 points
Group 4: Projected Points -> 5 points
Group 5: Original Points -> 1 points
Group 5: Projected Points -> 1 points
Group 6: Original Points -> 1 points
Group 6: Projected Points -> 1 points
Group 7: Original Points -> 1 points
Group 7: Projected Points -> 1 points
Group 8: Original Points -> 5 points
Group 8: Projected Points -> 5 points
Group 9: Original Points -> 2 points
Group 9: Projected Points -> 2 points
Group 10: Original Points -> 6 points
Group 10: Projected Points -> 6 points
Group 11: Original Points -> 2 points
Group 11: Projected Points -> 2 points
Group 12: Original Points -> 1 points
Group 12: Projected Points -> 1 points
Group 13: Original 